# PyIceberg with Unity Catalog

### Setup

In [0]:
%pip install pyiceberg

In [0]:
# Loading credentials in json file - not recommended. Secrets are better
import json

with open('creds.json', 'r') as file:
    creds = json.load(file)

### Authentication

In [0]:
from pyiceberg.catalog import load_catalog

UC_CATALOG = 'my_data'
UC_CREDENTIAL  = creds['token']
UC_DATABRICKS_URL = creds['url']
UC_CATALOG_URI = f'https://{UC_DATABRICKS_URL}/api/2.1/unity-catalog/iceberg-rest'


In [0]:
uc_catalog_properties = {
  'type':      'rest',
  'uri':       UC_CATALOG_URI,
  'token':     UC_CREDENTIAL,
  'warehouse': UC_CATALOG
}

uc_catalog = load_catalog(**uc_catalog_properties)

### List schemas (namespaces) and tables in the default schema

In [0]:

print(uc_catalog.list_namespaces())
print(uc_catalog.list_tables("default"))

In [0]:
table = uc_catalog.load_table("default.users_iceberg")
table

### Create a namespace (schema)

In [0]:
uc_catalog.create_namespace("data_created_from_pyiceberg")

In [0]:
print(uc_catalog.list_namespaces())

### Create a new Iceberg table in UC

In [0]:
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.table.sorting import SortOrder, SortField
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    FloatType,
    DoubleType,
    StringType,
    NestedField,
    StructType,
)

# Define a schema for the table
schema = Schema(
    NestedField(field_id=1, name="team", field_type=StringType(), required=True),
    NestedField(field_id=2, name="city", field_type=StringType(), required=True),
    NestedField(field_id=3, name="mascot", field_type=StringType(), required=False)
)

# Partition the table
partition_spec = PartitionSpec(
    PartitionField(
        source_id=1, field_id=1000, transform="day", name="datetime_day"
    )
)
 
# # Sort on the city
sort_order = SortOrder(SortField(source_id=2, transform='identity'))

uc_catalog.create_table(
    identifier="data_created_from_pyiceberg.sport_teams",
    schema=schema,
    partition_spec=partition_spec,
    sort_order=sort_order,
)

In [0]:
print(uc_catalog.list_tables("data_created_from_pyiceberg"))